In [ ]:
# Install requirements
!pip install -q transformers datasets peft bitsandbytes accelerate
!pip install -U transformers
!pip install -U datasets
!pip install -U peft bitsandbytes accelerate

# Imports
import torch
from datasets import Dataset
# import transformers first to get the latest version then import the rest
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training



# Upload your JSON
from google.colab import files
uploaded = files.upload()

dataset_path = list(uploaded.keys())[0]

import json
with open(dataset_path, "r") as f:
    data = json.load(f)

# Create HuggingFace Dataset
hf_dataset = Dataset.from_list([
    {"text": f"Question: {item['question']}\nAnswer: {item['answer']}"}
    for item in data
])

# Use  Llama 3B
model_name = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": True,
}

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto", # Let transformers choose the device
    trust_remote_code=True,
    **bnb_config
)
# Prepare for LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# Training Args
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,  # You can increase if you want
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    # Replace 'evaluation_strategy' with 'eval_strategy'
    eval_strategy="no",
    save_strategy="no",
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
)

# Trainer
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=lambda data: {'input_ids': torch.stack([torch.tensor(f["input_ids"]) for f in data]), # Convert input_ids to tensors before stacking
                                'attention_mask': torch.stack([torch.tensor(f["attention_mask"]) for f in data]), # Convert attention_mask to tensors before stacking
                                'labels': torch.stack([torch.tensor(f["input_ids"]) for f in data])}, # Convert labels to tensors before stacking
)

# Fine-tuning Start
trainer.train()

# Save fine-tuned model
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

print("✅ Training Done!")

# Test the model
from transformers import pipeline

pipe = pipeline("text-generation", model="./finetuned_model", tokenizer="./finetuned_model", device=0)

prompt = "Question: What is a Variable in JavaScript?\nAnswer:"
print(pipe(prompt, max_new_tokens=100))

Saving js_ontology_expanded_qa.json to js_ontology_expanded_qa (3).json


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Map:   0%|          | 0/620 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default)

Step,Training Loss
10,11.345800
20,1.364900
30,0.202500
40,0.166900
50,0.127200
60,0.095200
70,0.083200
80,0.079800
90,0.070900
100,0.061700


✅ Training Done!


Device set to use cuda:0


[{'generated_text': 'Question: What is a Variable in JavaScript?\nAnswer: A Variable is a special type of Object in JavaScript, inheriting its properties and methods.'}]


In [ ]:
model.save_pretrained("./drive/MyDrive/ColabNotebooks/finetuned_model")
tokenizer.save_pretrained("./drive/MyDrive/ColabNotebooks/finetuned_model")

('./drive/MyDrive/ColabNotebooks/finetuned_model/tokenizer_config.json',
 './drive/MyDrive/ColabNotebooks/finetuned_model/special_tokens_map.json',
 './drive/MyDrive/ColabNotebooks/finetuned_model/tokenizer.model',
 './drive/MyDrive/ColabNotebooks/finetuned_model/added_tokens.json',
 './drive/MyDrive/ColabNotebooks/finetuned_model/tokenizer.json')

In [ ]:
prompt = "Question: Write ONLY the JavaScript code for adding two numbers. No explanation. Just code.\nAnswer:"
print(pipe(prompt, max_new_tokens=100))

[{'generated_text': 'Question: Write ONLY the JavaScript code for adding two numbers. No explanation. Just code.\nAnswer: The JavaScript code for adding two numbers is:\nfunction add(a, b) {\n  return a + b;\n}\n'}]


In [ ]:
prompt = "Question: Write ONLY the JavaScript code for calculating subtraction. No explanation. Just code.\nAnswer:"
print(pipe(prompt, max_new_tokens=100))

[{'generated_text': 'Question: Write ONLY the JavaScript code for calculating subtraction. No explanation. Just code.\nAnswer: The JavaScript code for calculating subtraction is:\nfunction subtraction(a, b) {\n  return a - b;\n}\n'}]


In [ ]:
prompt = "Question: Create a for loop that prints numbers from 1 to 10.\nAnswer:"
print(pipe(prompt, max_new_tokens=100))

[{'generated_text': 'Question: Create a for loop that prints numbers from 1 to 10.\nAnswer: The question asks for a Create a for loop that prints numbers from 1 to 10.'}]
